###Trick to work with current code in repo
This add the location of the fc files in this repo to the system paths so it can be imported as normal

In [21]:
import os, sys
lib_path = os.path.abspath(os.path.join('..'))
sys.path.append(lib_path)

In [22]:
import openpyxl
import collections
def import_rows(workbook_name,
                worksheet_name):
    '''
    Access an excel doument and imports data from a specific spreadsheet.
    Returns a list of dictionaries, which contain key value pairs of the data
    *  The spreadsheet should contain a header row with the names of each key.
    *  Each subsequent row then contains the values contained in a single dictionary.
    *  Header row keys are parsed from A1 until a column without a entry is reached.
    *  Dictionaries are parsed until a row is reached which as no values for any key.
        This means that empty values for a key are acceptable for a row
    '''
    rows_data = []
    # Load workbook and sheet
    wb = openpyxl.load_workbook(workbook_name)
    if 'cells' not in wb:
        raise IOError('No sheet named cells! Name the worksheet used to import data cells please')
    ws = wb.get_sheet_by_name(worksheet_name)
    for r, row in enumerate(ws.rows):
        #Import headers
        if r==0:
            headers = []
            c = 0
            while ws.cell(row = r, column = c).value  != None:
                headers.append('_'.join(str(ws.cell(row = r, column = c).value).split()).lower())
                c+=1
        #Import row
        else:
            row_data = collections.OrderedDict()
            #Import data from a row
            for c, key in enumerate(headers):
                value = str(ws.cell(row = r, column = c).value)
                #Raise an error if a formula is used
                if value != None and isinstance(value, basestring) and value[0] == '=':
                    raise ImportError("Error: Ran into an excel formula, use plain text only. ("+str(r)+', '+str(c)+')')
                row_data[key] = value
            #If row was empty breakt the for loop
            if sum([v != None for k, v in row_data.iteritems()]) == 0:
                print [v is None for k, v in row_data.iteritems()]
                break;
            else:
                rows_data.append(row_data)
    return rows_data

def export_worksheet(workbook_name,
                worksheet_name,
               data):
    '''
    Writes a list of lists to a excel file
    Overwrite cells but not worksheets
    The first dimension is associated with the column, the second
    dimension is associated with the row
    '''
    #Open Workbook
    if os.path.exists(workbook_name):
        wb = openpyxl.load_workbook(workbook_name)
    else:
        wb = openpyxl.Workbook()
    if wb.get_sheet_by_name(worksheet_name) is None:
        wb.create_sheet().title = worksheet_name
    ws = wb.get_sheet_by_name(worksheet_name)
    
    for c, column_vector in enumerate(data):
        for r, value in enumerate(column_vector):
            ws.cell(row = r, column = c).value = str(value)
    
    try:
        wb.save(workbook_name)
    except IOError:
        raise IOError("Excel document must be closed! Please save and close it and run this script again!")

In [23]:
import matplotlib.pyplot as plt
import fc.plot
import gc
def gate_plot(ungated_points, gated_points, contour, title='', filename=None):
    '''
    Plots both the FSC SSC scatterplot with gate, and the overlay of channel one gated and ungated histograms
    Used to confirm automated gating was approriate for each sample
    '''
    fig = plt.figure(figsize=(12,9))
    density_ax = fig.add_subplot(2,1,1)
    
    fc.plot.density2d(ungated_points, gate = contour, sigma=2.5, ax = density_ax, colorbar=False,xlabel='FSC-H',ylabel='SSC-H')
    density_ax.set_aspect('auto')
    plt.title(title)
    
    flour_ax = fig.add_subplot(2,1,2)
    fc.plot.hist1d(ungated_points[:,2],ax = flour_ax,edge_color=(0.5, 0.85, 0.3), face_color=(0.8, 0.95, 0.7))
    fc.plot.hist1d(gated_points[:,2],ax = flour_ax, edge_color=(0.2, 0.7, 0), face_color=(0.6, 0.9, 0.4),xlabel='FL1-H')
    plt.tight_layout()
    
    if filename != None:
        filename = plot_gated_folder + '/gated_%03d.png'%(i+1)
        plt.savefig(filename, bbox_inches='tight')
    
    plt.close()
    fig.clf()
    gc.collect();

In [24]:
import numpy
def generate_stats(samples):
    '''
    Takes a set of samples, generate stats for them
    Per channel stats:
    -  Mean
    -  Standard deviation
    -  CV
    -  Mode
    
    More stats can be added in the future
    '''
    def function_specifier(fn, channel=0, attribute=''):
        def fn_wrapper(data): #take any arguments
            return fn(getattr(sample, attribute, sample), channel) #pass any arguments to fn()
        return fn_wrapper

    def counts(data, channel):
        return data[:,channel].size
    
    def mean(data, channel):
        return numpy.mean(data[:,channel])

    def mode(data, channel):
        return numpy.argmax(numpy.bincount(data[:,channel].astype('int32')))

    def std(data, channel):
        return numpy.std(data[:,channel])

    def CV(data, channel):
        return numpy.std(data[:,channel])/numpy.mean(data[:,channel])
    
    def median(data, channel):
        return numpy.median(data[:,channel])

    def iqr(data, channel):
        q75, q25 = numpy.percentile(data[:,channel], [75 ,25])
        return q75 - q25
    
    def RCV(data, channel):
        q75, q25 = numpy.percentile(data[:,channel], [75 ,25])
        return numpy.median(data[:,channel])/(q75 - q25)
    
    stat_functions = []

    stat_functions.append(('Ungated Counts',function_specifier(counts)))
    stat_functions.append(('Gated Counts',function_specifier(counts, attribute = 'data_transformed')))
    
    for index, name in [(2, 'FL1-H'), (3, 'FL2-H'), (4, 'FL3-H')]:
        stat_functions.append((name+' Mean',function_specifier(mean, index, 'data_transformed')))
        stat_functions.append((name+' Mode',function_specifier(mode, index, 'data_transformed')))
        stat_functions.append((name+' Std',function_specifier(std, index, 'data_transformed')))
        stat_functions.append((name+' CV',function_specifier(CV, index, 'data_transformed')))
        stat_functions.append((name+' Median',function_specifier(median, index, 'data_transformed')))
        stat_functions.append((name+' IQR',function_specifier(iqr, index, 'data_transformed')))
        stat_functions.append((name+' RCV',function_specifier(RCV, index, 'data_transformed')))

    output = []
    
    for header in samples[0].metadata.keys():
        output.append([header])
    for name, stat_function in stat_functions:
        output.append([name])
        
    for sample in samples:
        i = 0
        # Add meta data to output array
        for header, value in sample.metadata.iteritems():
            if value == None:
                value = ''
            output[i].append(value)
            i+=1
        for name, stat_function in stat_functions:
            value = stat_function(sample.data_transformed)
            output[i].append(value)
            i+=1
    return output

In [25]:
import fc.gate
def auto_gate(sample):
    '''
    * Gates out first and last channel of all 5 channels
    * Gates out first 250 events and last 100 events of run
    * Performs a density gate on remaining data
    * Transforms data to an exponential form
    '''
    gate_fraction = sample.metadata['gate_fraction'] if 'gate_fraction' in sample.metadata and sample.metadata['gate_fraction'] != None else 0.1
    sigma = sample.metadata['sigma'] if 'sigma' in sample.metadata and sample.metadata['sigma'] != None else 20
    
    high_low_gate = fc.gate.high_low(sample, high=(2**10)-1, low=0)
    start_stop_gate = fc.gate.start_stop(sample, num_start=250, num_stop=100)
    sample.data_coarse = sample[high_low_gate*start_stop_gate]
    
    density2d_gate, sample.density2d_contour = fc.gate.density2d(sample.data_coarse,gate_fraction = gate_fraction,sigma = sigma)
    sample.data_fine = sample.data_coarse[density2d_gate]
    
    sample.data_transformed = fc.transform.exponentiate(sample.data_fine)

In [30]:
import fc.gate, fc.transform, fc.io, os
from Tkinter import Tk
from tkFileDialog import askopenfilename



# Import bead metadata
# Load bead data
# Analysis bead data
# Graph bead data
# Import cell metadata

#Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
#import_path = askopenfilename() # show an "Open" dialog box and return the path to the selected file
import_path = "..\\..\\..\\Data\\Cytometry\\20150615\\experiment.xlsm"
directory = os.path.dirname(os.path.realpath(import_path))
print "Loading files..."
# Load Files
samples = []
for row in import_rows(import_path,"cells"):
    file_path = str(row['file_path'])
    if file_path[-3] == '.':
        file_path = file_path+'0'
    sample = fc.io.TaborLabFCSData(directory+'\\'+file_path)
    sample.metadata = row
    samples.append(sample)
print "Generating gate for file: ",
# Generate Gates
for i, sample in enumerate(samples):
    print str(i+1)+', ',
    auto_gate(sample)
print ''
    
# Plot graphs of gating
print "Generating plots for file: ",
plot_gated_folder = directory + '\\plot_gated'
if not os.path.exists(plot_gated_folder):
    os.makedirs(plot_gated_folder)
for i, sample in enumerate(samples):
    print str(i+1)+', ',
    gate_plot(sample.data_coarse, sample.data_fine, sample.density2d_contour, title = sample.metadata['file_path'], filename = plot_gated_folder+'/gated_%03d.png'%(i+1))
print ''

# Generate stats
print 'Generating stats...'
output = generate_stats(samples)
export_worksheet(import_path,"stats",output)
print 'All done!'  

Loading files...
Generating gate for file:  1,  2,  3,  4,  5,  6,  7,  8,  9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  
Generating plots for file:  1,  2,  3,  4,  5,  6,  7,  8,  9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  
Generating stats...
All done!
